<h1 align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazirmatn" color="#0099cc">
انتخاب ویژگی
</font>
</h1>

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazirmatn" color="#0099cc">
مجموعه‌داده
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazirmatn" size=3>
در این مثال از مجموعه‌داده‌ی MNIST استفاده می‌کنیم که می‌توان آن را مستقیماً به کمک کتابخانه‌ی <code>scikit-learn</code> بارگیری کرد.
</p>

In [1]:
from sklearn.datasets import fetch_openml

# loading the mnist dataset
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=True)

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazirmatn" color="#0099cc">
آستانه‌ی واریانس (Variance Threshold)
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazirmatn" size=3>
برای این کار می‌توانیم از تابع <code>sklearn.feature_selection.VarianceThreshold</code> استفاده کنیم. این تابع ویژگی‌هایی را که واریانس مجموعه آموزشی‌شان کمتر از مقدار تنظیم‌شده برای <code>threshold</code> باشد حذف می‌کند. به‌صورت پیش‌فرض این آرگومان معادل <code>0.0</code> تنظیم می‌شود که بدین معنی‌ست ویژگی‌های با واریانس صفر (مقدار ویژگی در تمام نمونه‌ها مشابه و ثابت است) حذف شوند.
در کد زیر سعی داریم از تکنیک <code>varianceThreshold</code> استفاده کرده و ویژگی‌هایی که واریانس‌شان کمتر از <code>0.01</code> است را حذف کنیم. تنها توجه داشته باشید که پیش از این کار باید ویژگی‌ها را نیز هم‌مقیاس کنیم تا تصمیم‌گیری‌مان عادلانه باشد.
</p>

In [2]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import MinMaxScaler

# feature scaling
scaler = MinMaxScaler()
scaled = scaler.fit_transform(X)

# feature selection
selector = VarianceThreshold(threshold=0.01)
selected = selector.fit_transform(scaled)

print('Number of features before selection:', X.shape[1])
print('Number of features after selection:', selected.shape[1])

Number of features before selection: 784
Number of features after selection: 443


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazirmatn" color="#0099cc">
تحلیل تک‌متغیره (Univariate analysis)
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazirmatn" size=3>
 تابع <code>SelectKBest</code>، عدد <code>k</code> را می‌گیرد و با توجه به آزمون آماری تعیین‌شده، <code>k</code> تا از بهترین ویژگی‌ها از نظر ارتباط با ویژگی هدف را برمی‌گرداند.
 برای آزمون‌های آماری نیز می‌توانیم از توابع موجود در کتابخانه‌ی  <code>scikit-learn</code> همچون آزمون <code>chi2</code> کمک بگیریم.
 در مثال زیر قصد داریم براساس آزمون آماری <code>chi2</code>، تنها <code>32</code>تا از مرتبط‌ترین ویژگی‌ها با متغیر هدف را پیدا کرده و نگه داریم.
 توجه شود از آنجا که آزمون آماری <code>chi2</code> تنها مقادیر غیرمنفی را می‌پذیرد این‌بار از تغییر مقیاس با روش <code>MinMaxScaler</code> استفاده کرده‌ایم. البته در این مجموعه‌داده‌ی خاص مقادیر از ابتدا غیرمنفی هستند اما نیاز است که به این نکته توجه داشته باشید.
</p>

In [3]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, chi2

# feature scaling
scaler = MinMaxScaler()
scaled = scaler.fit_transform(X)

# feature selection
selector = SelectKBest(chi2, k=32)
selected = selector.fit_transform(scaled, y)

print('Number of features before selection:', X.shape[1])
print('Number of features after selection:', selected.shape[1])

Number of features before selection: 784
Number of features after selection: 32


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazirmatn" color="#0099cc">
هم‌بستگی جفت جفت ویژگی‌ها (Feature pairwise correlation)
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazirmatn" size=3>
    این تکنیک مستقیماً در <code>scikit-learn</code> پیاده‌سازی نشده، اما خودمان می‌توانیم در چند خط کد پیاده‌اش کنیم.
    تابع <code dir=ltr>corr()</code> می‌تواند به‌راحتی هم‌بستگی جفت ویژگی‌ها را برای‌مان محاسبه کند. البته از آنجا که به دنبال میزان هم‌بستگی هستیم و علامت عدد برای‌مان اهمیت ندارد از <code dir=ltr>abs()</code> استفاده می‌کنیم تا مقادیر منفی نیز به مثبت تبدیل شوند.
    توجه داشته باشید که ماتریس هم‌بستگی یک ماتریس متقارن است و عناصر تکراری دارد، بنابراین می‌توانیم تنها بخش بالا مثلثی آن را نگه داریم. پس از این کار کافیست هر کدام از ستون‌ها (ویژگی‌ها) را بررسی کنیم و اگر مقدار هم‌بستگی آن با حداقل یکی از ویژگی‌های دیگر بیشتر از حد آستانه بود، می‌توانیم آن ویژگی را حذف کنیم.
</p>

In [4]:
import numpy as np

# computing the absolute values of correlation
corr_matrix = X.corr().abs()

# keeping only upper part of correlation matrix
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# finding the highly correlated features
THRESHOLD = 0.90
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > THRESHOLD)]
print('Feature(s) to drop:', ', '.join(to_drop))

# dropping the highly correlated features
selected = X.drop(to_drop, axis=1)

print('Number of features before selection:', X.shape[1])
print('Number of features after selection:', selected.shape[1])

Feature(s) to drop: pixel16, pixel87, pixel225, pixel281, pixel421, pixel617, pixel644, pixel646, pixel727
Number of features before selection: 784
Number of features after selection: 775
